In [11]:
import pandas as pd
import random
import authorization
import numpy as np
from numpy.linalg import norm

In [12]:
df = pd.read_csv("valence_arousal_dataset.csv")
print(df.shape)
df.head()

(5841, 6)


,id,genre,track_name,artist_name,valence,energy
0,0O36Yqb2aFcThBphczYoAY,acoustic,A Drop In The Ocean,Ron Pope,0.473,0.452
1,2izoqbHZCGaoFOO8vcc1hK,acoustic,Boyfriend,Alex G,0.741,0.737
2,7IByJvSqRFltGyiiIiL4wn,acoustic,Who You Love (feat. Katy Perry),John Mayer,0.691,0.351
3,265Anh9hGoozFigjUVLUeD,acoustic,New Shoes,Paolo Nutini,0.854,0.735
4,1SZ63wN0pk18Dr4Epyhcsf,acoustic,Tainted Love,Hannah Peel,0.437,0.192


In [13]:
df["mood_vec"] = df[["valence", "energy"]].values.tolist()
df["mood_vec"].head()

0    [0.473, 0.452]
1    [0.741, 0.737]
2    [0.691, 0.351]
3    [0.854, 0.735]
4    [0.437, 0.192]
Name: mood_vec, dtype: object

In [5]:
sp = authorization.authorize() 

In [14]:
def recommend(track_id, ref_df, sp, n_recs = 5):
    
    # Crawl valence and arousal of given track from spotify api
    track_features = sp.track_audio_features(track_id)
    track_moodvec = np.array([track_features.valence, track_features.energy])
    print(f"mood_vec for {track_id}: {track_moodvec}")
    
    # Compute distances to all reference tracks
    ref_df["distances"] = ref_df["mood_vec"].apply(lambda x: norm(track_moodvec-np.array(x)))
    # Sort distances from lowest to highest
    ref_df_sorted = ref_df.sort_values(by = "distances", ascending = True)
    # If the input track is in the reference set, it will have a distance of 0, but should not be recommendet
    ref_df_sorted = ref_df_sorted[ref_df_sorted["id"] != track_id]
    
    # Return n recommendations
    return ref_df_sorted.iloc[:n_recs]

In [15]:
track1 = random.choice(df["id"])
recommend(track_id = track1, ref_df = df, sp = sp, n_recs = 5)

mood_vec for 6vxHp3CDNo0afgKGp2yi1E: [0.798 0.816]


,id,genre,track_name,artist_name,valence,energy,mood_vec,distances
3254,5cY6DFEsxpChho3JU7AG0h,k-pop,The Boys - Clinton Sparks & Disco Fries Remix,Girls' Generation,0.793,0.816,"[0.793, 0.816]",0.005000
1776,278KHzLuQwzsU666grEiB8,forro,"Bola de Meia, Bola de Gude",Menina do Céu,0.804,0.813,"[0.804, 0.813]",0.006708
4282,0V7oNGTXb8yLaMaLRnCpPR,power-pop,All at Once,The Orange Peels,0.798,0.824,"[0.798, 0.824]",0.008000
4933,6A3xXylm8EhTuxOOzMN26D,salsa,La Habana Me Llama,Manolito Simonet y su Trabuco,0.805,0.824,"[0.805, 0.824]",0.010630
1213,3Jsfr8rhZF37o56Hu4qXYB,deep-house,Vanishing Point - Original Mix,Dusky,0.806,0.806,"[0.806, 0.806]",0.012806


In [16]:
classical = "7etGs69UCKGxJIHxKFe7BD"
recommend(track_id = classical, ref_df = df, sp = sp, n_recs = 5)

mood_vec for 7etGs69UCKGxJIHxKFe7BD: [0.186 0.183]


,id,genre,track_name,artist_name,valence,energy,mood_vec,distances
5114,6EB1urn3bHjTXjkT9twKxZ,singer-songwriter,"John Wayne Gacy, Jr.",Sufjan Stevens,0.183,0.179,"[0.183, 0.179]",0.005000
811,2bbhyUWJ5VjdfI3P4PRLu2,chill,Samson,Regina Spektor,0.184,0.193,"[0.184, 0.193]",0.010198
3863,5mHTg6cqhyP0BmsTQKzN1X,new-release,Sprained Ankle,Julien Baker,0.195,0.174,"[0.195, 0.174]",0.012728
3927,2OP6tdFv9lvOCvOx2nHVOw,opera,"Gounod: Roméo et Juliette, Act 1: ""Je veux viv...",Maria Callas,0.189,0.197,"[0.189, 0.197]",0.014318
3890,4OcvKUhccmfqbCU8552o81,opera,"Acis and Galatea / Act 2: ""I rage, I melt, I b...",George Frideric Handel,0.189,0.168,"[0.189, 0.168]",0.015297


In [17]:
lofi = "1cWxCu1NxsY4NGzdHSwpMl"
recommend(track_id = lofi, ref_df = df, sp = sp, n_recs = 5)

mood_vec for 1cWxCu1NxsY4NGzdHSwpMl: [0.103 0.304]


,id,genre,track_name,artist_name,valence,energy,mood_vec,distances
5517,4PRQ8IJzRvbDWNONJWGbwH,synth-pop,Nightcall,London Grammar,0.1010,0.306,"[0.101, 0.306]",0.002828
2000,3X5P8ZCyytPL91vLfI5dbe,gospel,His Strength Is Perfect,CeCe Winans,0.1020,0.288,"[0.102, 0.288]",0.016031
5321,2HIqNbnpN41Xws8Tbm5ied,soundtracks,"Main Title (From ""Star Wars"")",John Williams,0.1080,0.320,"[0.108, 0.32]",0.016763
2184,2jeBcSclYZHOGQ89ZyF2BN,grunge,Lizzy,Melvins,0.0894,0.294,"[0.0894, 0.294]",0.016881
3721,6zJms3MX11Qu1IKF44LoRW,movies,Us,Regina Spektor,0.1200,0.305,"[0.12, 0.305]",0.017029


In [18]:
rock = "4u7EnebtmKWzUH433cf5Qv"
recommend(track_id = rock, ref_df = df, sp = sp, n_recs = 5)

mood_vec for 4u7EnebtmKWzUH433cf5Qv: [0.224 0.404]


,id,genre,track_name,artist_name,valence,energy,mood_vec,distances
3845,14c6Q17DoHUY4A2u9HlxxL,new-release,Hyde,Astrid S,0.220,0.404,"[0.22, 0.404]",0.004000
4567,48s4feNgV243gbAGTTBIrc,rainy-day,Don't You Remember,Adele,0.226,0.400,"[0.226, 0.4]",0.004472
2229,2GAIycsMaDVtMtdvxzR2xI,guitar,Bohemian Rhapsody - Remastered 2011,Queen,0.228,0.402,"[0.228, 0.402]",0.004472
3503,40y9N4E7vOXUQpt5fLbcfY,mandopop,熒光,Zhang Yao,0.226,0.397,"[0.226, 0.397]",0.007280
5064,78ZBoqAkOJBOqYesivkQOI,show-tunes,The Journey To The Heaviside Layer - UK 1981 /...,Andrew Lloyd Webber,0.226,0.412,"[0.226, 0.412]",0.008246
